In [ ]:
!pip install phe

In [ ]:
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from phe import paillier
import base64
from functools import reduce

# ------------------ SETUP PHASE ------------------

In [ ]:
print("[Setup Phase] Generating keys...")

# Election authority generates Paillier keypair for homomorphic encryption
public_key, private_key = paillier.generate_paillier_keypair()

# Simulate registration of 3 voters, each with RSA keypair for signing
voter_keys = []
for i in range(3):
    private = rsa.generate_private_key(public_exponent=65537, key_size=2048)
    public = private.public_key()
    voter_keys.append((private, public))
    print(f"Voter {i+1} registered with RSA keypair.")

[Setup Phase] Generating keys...
Voter 1 registered with RSA keypair.
Voter 2 registered with RSA keypair.
Voter 3 registered with RSA keypair.


# ------------------ VOTING PHASE ------------------

In [ ]:
print("\n[Voting Phase] Voters cast encrypted and signed votes...")

# Simulated votes (1 for YES, 0 for NO)
votes = [1, 0, 1]  # Voter 1: YES, Voter 2: NO, Voter 3: YES
encrypted_votes = []
signed_votes = []

for i, vote in enumerate(votes):
    # Encrypt the vote using Paillier
    enc_vote = public_key.encrypt(vote)
    encrypted_votes.append(enc_vote)

    # Sign the vote using voter's RSA private key
    signer = voter_keys[i][0]
    signature = signer.sign(
        str(vote).encode(),
        padding.PSS(mgf=padding.MGF1(hashes.SHA256()), salt_length=padding.PSS.MAX_LENGTH),
        hashes.SHA256()
    )
    signed_votes.append((enc_vote, signature))
    print(f"Voter {i+1} cast vote: {vote} (encrypted and signed)")


[Voting Phase] Voters cast encrypted and signed votes...
Voter 1 cast vote: 1 (encrypted and signed)
Voter 2 cast vote: 0 (encrypted and signed)
Voter 3 cast vote: 1 (encrypted and signed)


# ------------------ VERIFICATION PHASE ------------------

In [ ]:
print("\n[Verification Phase] Verifying signatures of votes...")

def verify_vote(index, enc_vote, signature):
    verifier = voter_keys[index][1]
    try:
        verifier.verify(
            signature,
            str(votes[index]).encode(),
            padding.PSS(mgf=padding.MGF1(hashes.SHA256()), salt_length=padding.PSS.MAX_LENGTH),
            hashes.SHA256()
        )
        print(f"Vote {index+1} signature verified.")
        return True
    except Exception as e:
        print(f"Vote {index+1} failed verification: {e}")
        return False

valid_votes = []
for i, (enc_vote, sig) in enumerate(signed_votes):
    if verify_vote(i, enc_vote, sig):
        valid_votes.append(enc_vote)


[Verification Phase] Verifying signatures of votes...
Vote 1 signature verified.
Vote 2 signature verified.
Vote 3 signature verified.


# ------------------ TALLYING PHASE ------------------

In [ ]:


print("\n[Tallying Phase] Aggregating and decrypting votes...")

# Homomorphically add all valid encrypted votes
total_encrypted = reduce(lambda x, y: x + y, valid_votes)

# Decrypt final result
final_tally = private_key.decrypt(total_encrypted)

print(f"\nFinal Tally: {final_tally} YES vote(s) out of {len(votes)} total votes")


[Tallying Phase] Aggregating and decrypting votes...

Final Tally: 2 YES vote(s) out of 3 total votes
